<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.3: Higher-Order Functions
**Prev: [Interlude: Chisel Standard Library](3.2_interlude.ipynb)**<br>
**Next: [Functional Programming](3.4_functional_programming.ipynb)**

## Motivation
이전 모듈의 성가신 `for` 루프는 장황하고 함수형 프로그래밍의 목적을 무너뜨립니다! 이 모듈에서 생성기는 funct-ky를 얻습니다.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml


path: String = "/home/parkdongho/dev/chisel-bootcamp-kr/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

---
# A Tale of Two FIRs <a name="compact-fir"></a>
마지막 모듈에서 FIR 필터의 컨볼루션 부분은 다음과 같이 작성되었습니다.

```scala
val muls = Wire(Vec(length, UInt(8.W)))
for(i <- 0 until length) {
  if(i == 0) muls(i) := io.in * io.consts(i)
  else       muls(i) := regs(i - 1) * io.consts(i)
}

val scan = Wire(Vec(length, UInt(8.W)))
for(i <- 0 until length) {
  if(i == 0) scan(i) := muls(i)
  else scan(i) := muls(i) + scan(i - 1)
}

io.out := scan(length - 1)
```

요약하자면 'io.in'의 각 요소를 'io.consts'의 해당 요소와 곱하여 'muls'에 저장하는 것입니다.
그런 다음 `muls`의 요소는 `scan`에 누적되며, `scan(0) = muls(0)`, `scan(1) = scan(0) + muls(1) = muls(0) + muls (1)`, 그리고 일반적으로 `scan(n) = scan(n-1) + muls(n) = muls(0) + ... + muls(n-1) + muls(n)`.
`scan`의 마지막 요소(모든 `muls`의 합계와 동일)는 `io.out`에 할당됩니다.

그러나 매우 간단한 작업으로 간주될 수 있는 작업에 대해서는 매우 장황합니다. 사실 한 줄로 작성할 수 있는 모든 내용은 다음과 같습니다.

```scala
io.out := (taps zip io.consts).map { case (a, b) => a * b }.reduce(_ + _)
```

뭐하는거야?! 분해해 보겠습니다.
- `taps`가 `taps(0) = io.in`, `taps(1) = regs(0)` 등의 모든 샘플 목록이라고 가정합니다.
- `(taps zip io.consts)`는 `taps`와 `io.consts`라는 두 개의 목록을 가져와 각 요소가 해당 위치의 입력에 있는 요소의 튜플인 하나의 목록으로 결합합니다. 구체적으로, 그 값은 `[(taps(0), io.consts(0)), (taps(1), io.consts(1)), ..., (taps(n), io.consts( n))]`. 마침표는 선택 사항이므로 `(taps.zip(io.const))`와 동일합니다.
- `.map { case (a, b) => a * b }`는 익명 함수(두 요소의 튜플을 취하여 결과를 반환함)를 list의 요소에 적용하고 결과를 반환합니다. 이 경우 결과는 자세한 예의 `muls`와 동일하며 값은 `[taps(0) * io.consts(0), taps(1) * io.consts(1), ... , taps(n) * io.consts(n)]`. 다음 모듈에서 익명 함수를 다시 살펴볼 것입니다. 지금은 이 구문을 배우기만 하면 됩니다.
- 마지막으로 `.reduce(_ + _)`는 list의 요소에 함수(addition of elements)도 적용합니다. 그러나 두 개의 인수가 필요합니다. 첫 번째는 현재 accumulation이고 두 번째는 list 요소입니다(첫 번째 반복에서는 처음 두 요소만 추가함). 괄호 안에 두 개의 밑줄로 표시됩니다. 그러면 결과는 왼쪽에서 오른쪽 순회를 가정할 때 `(((muls(0) + muls(1)) + muls(2)) + ...) + muls(n)`이 됩니다. 더 깊은 중첩 괄호가 먼저 평가됩니다. 이것은 컨볼루션의 출력입니다.

---
# Functions as Arguments
공식적으로 `map` 및 `reduce`와 같은 함수를 _고차 함수(higher-order functions)_ 라고 합니다. 함수를 인수로 사용하는 함수입니다.
위의 예에서 볼 수 있듯이 이것은 일반적인 계산 패턴을 캡슐화하는 매우 강력한 구조로, 흐름 제어(flow control) 대신 응용 프로그램 논리에 집중할 수 있도록 하여 매우 간결한 코드를 생성합니다.

## Different ways of specifying functions
위의 예에서 함수를 지정하는 두 가지 방법이 있음을 눈치채셨을 것입니다.
- 각 인수가 정확히 한 번 참조되는 함수의 경우 밑줄(`_`)을 사용하여 각 인수를 참조할 수 있습니다. 위의 예에서 `reduce` 인수 함수는 두 개의 인수를 취했으며 `_ + _`로 지정할 수 있습니다. 편리하긴 하지만 여기에는 추가 신비한 규칙이 적용되므로 작동하지 않으면 다음을 시도해 보세요.
- 입력 인수 목록을 명시적으로 지정합니다. reduce는 `(a, b) => a + b`로 명시적으로 작성될 수 있으며, 인수 목록을 괄호 안에 넣고 `=>` 다음에 해당 인수를 참조하는 함수 본문이 오는 일반적인 형식을 사용합니다.
- tuple unpacking이 필요할 때 `case (a, b) => a * b`와 같이 `case` 문을 사용합니다. 그것은 단일 인수, 두 요소의 튜플을 취하고 변수 `a`와 `b`로 압축을 푼 다음 함수 본문에서 사용할 수 있습니다.

## Practice in Scala
마지막 모듈에서는 `List`와 같은 Scala Collections API의 주요 클래스를 보았습니다.
이러한 고차 함수는 이러한 API의 일부이며 실제로 위의 예는 `List`에서 `map` 및 `reduce` API를 사용합니다.
이 섹션에서는 예제와 연습을 통해 이러한 방법에 익숙해질 것입니다.
이 예에서는 단순성과 명확성을 위해 스칼라 숫자(`Int`)로 작업하지만 Chisel 연산자가 유사하게 동작하기 때문에 개념을 일반화해야 합니다.

<span style="color:blue">**Example: Map**</span><br>
`List[A].map`에는 type signature `map[B](f: (A) ⇒ B): List[B]` 가 있습니다. 이후 모듈에서 타입에 대해 자세히 알아볼 것입니다. 지금은 타입 A와 B를 `Int` 또는 `UInt`로 생각하십시오. 즉, 소프트웨어 또는 하드웨어 타입일 수 있습니다.

일반 영어에서는 `(f: (A) ⇒ B)` 타입의 인수를 취하거나 `A` 타입(입력 목록의 요소와 동일한 타입)의 인수 하나를 취하여 값을 반환하는 함수 타입 `B`(무엇이든 될 수 있음). 그런 다음 `map`은 `B` 타입(인수 함수의 반환 타입)의 새 목록을 반환합니다.

FIR 예제에서 List의 동작을 이미 설명했으므로 예제와 연습으로 바로 들어가 보겠습니다.

In [3]:
println(List(1, 2, 3, 4).map(x => x + 1))  // explicit argument list in function
println(List(1, 2, 3, 4).map(_ + 1))  // equivalent to the above, but implicit arguments
println(List(1, 2, 3, 4).map(_.toString + "a"))  // the output element type can be different from the input element type

println(List((1, 5), (2, 6), (3, 7), (4, 8)).map { case (x, y) => x*y })  // this unpacks a tuple, note use of curly braces

// Related: Scala has a syntax for constructing lists of sequential numbers
println(0 to 10)  // to is inclusive , the end point is part of the result
println(0 until 10)  // until is exclusive at the end, the end point is not part of the result

// Those largely behave like lists, and can be useful for generating indices:
val myList = List("a", "b", "c", "d")
println((0 until 4).map(myList(_)))

List(2, 3, 4, 5)
List(2, 3, 4, 5)
List(1a, 2a, 3a, 4a)
List(5, 12, 21, 32)
Range 0 to 10
Range 0 until 10
Vector(a, b, c, d)


myList: List[String] = List("a", "b", "c", "d")

<span style="color:red">**Exercise: Map**</span><br><a name="map-exercise"></a>

In [4]:
// Now you try: 
// Fill in the blanks (the ???) such that this doubles the elements of the input list.
// This should return: List(2, 4, 6, 8)
println(List(1, 2, 3, 4).map( ??? ))

: 

<span style="color:blue">**Example: zipWithIndex**</span><br>
`List.zipWithIndex`에는 `zipWithIndex: List[(A, Int)]` 타입 서명이 있습니다.

인수를 사용하지 않지만 각 요소가 원래 요소의 튜플인 목록과 인덱스(첫 번째 요소는 0임)를 반환합니다.
따라서 `List("a", "b", "c", "d").zipWithIndex`는 `List(("a", 0), ("b", 1), ("c", 2 ), ("d", 3))`

이것은 어떤 작업에서 요소 인덱스가 필요할 때 유용합니다.

이것은 매우 간단하기 때문에 몇 가지 예만 들어보겠습니다.

In [ ]:
println(List(1, 2, 3, 4).zipWithIndex)  // note indices start at zero
println(List("a", "b", "c", "d").zipWithIndex)
println(List(("a", "b"), ("c", "d"), ("e", "f"), ("g", "h")).zipWithIndex)  // tuples nest

<span style="color:blue">**Example: Reduce**</span><br>
`List[A].map`은 `reduce(op: (A, A) ⇒ A): A`와 유사한 타입 서명을 가지고 있습니다. (실제로는 더 관대합니다. `A`는 List 타입의 상위 타입이어야 하지만 여기서는 해당 구문을 다루지 않겠습니다.)

위에서도 설명했듯이 다음은 몇 가지 예입니다.

In [14]:
println(List(1, 2, 3, 4).reduce((a, b) => a + b))  // returns the sum of all the elements
println(List(1, 2, 3, 4).reduce(_ * _))  // returns the product of all the elements
println(List(1, 2, 3, 4).map(_ + 1).reduce(_ + _))  // you can chain reduce onto the result of a map

10
24
14


In [6]:
// Important note: reduce will fail with an empty list
println(List[Int]().reduce(_ * _))

: 

<span style="color:red">**Exercise: Reduce**</span><br><a name="reduce-exercise"></a>

In [13]:
// Now you try: 
// Fill in the blanks (the ???) such that this returns the product of the double of the elements of the input list.
// This should return: (1*2)*(2*2)*(3*2)*(4*2) = 384
println(List(1, 2, 3, 4).map(???).reduce(???))

: 

<span style="color:blue">**Example: Fold**</span><br>
`List[A].fold`는 초기 누적 값을 지정할 수 있다는 점을 제외하면 reduce와 매우 유사합니다.
`fold(z: A)(op: (A, A) ⇒ A): A`와 유사한 타입 서명을 가지고 있습니다. (`reduce`와 같이 `A` 타입도 더 관대합니다)

특히, 두 개의 인수 목록이 필요합니다. 첫 번째(`z`)는 초기 값이고 두 번째는 누적 함수입니다.
`reduce`와 달리 빈 목록으로 실패하지 않고 초기 값을 직접 반환합니다.

다음은 몇 가지 예입니다.

In [8]:
println(List(1, 2, 3, 4).fold(0)(_ + _))  // equivalent to the sum using reduce
println(List(1, 2, 3, 4).fold(1)(_ + _))  // like above, but accumulation starts at 1
println(List().fold(1)(_ + _))  // unlike reduce, does not fail on an empty input

10
11
1


<span style="color:red">**Exercise: Fold**</span><br><a name="fold-exercise"></a>

In [12]:
// Now you try: 
// Fill in the blanks (the ???) such that this returns the double the product of the elements of the input list.
// This should return: 2*(1*2*3*4) = 48
// Note: unless empty list tolerance is needed, reduce is a much better fit here.
println(List(1, 2, 3, 4).fold(???)(???))

: 

<span style="color:red">**Exercise: Decoupled Arbiter**</span><br>
이제 모든 것을 하나의 운동으로 통합해 봅시다.

이 예에서 우리는 _n_개의 분리된 입력과 하나의 분리된 출력을 가진 모듈인 분리된 arbiter를 만들 것입니다.
arbiter는 유효한 가장 낮은 채널을 선택하여 출력으로 전달합니다.

몇 가지 힌트:
- 건축학적으로:
   - 입력 중 하나라도 유효하면 `io.out.valid`는 true입니다.
   - 선택한 채널의 내부 배선을 고려하십시오.
   - 출력이 준비되고 해당 채널이 선택되면 각 입력의 'ready'는 true입니다(이는 준비와 유효를 조합하여 연결하지만 지금은 무시하겠습니다...)
- 다음 구성이 도움이 될 수 있습니다.
   - `map`, 특히 하위 요소의 Vec을 반환하는 경우, 예를 들어 `io.in.map(_.valid)`는 입력 번들의 유효한 신호 목록을 반환합니다.
   - `PriorityMux(List[Bool, Bits])`는 유효한 신호 및 비트 목록을 가져와 유효한 첫 번째 요소를 반환합니다.
   - UInt로 인덱싱하여 Vec에 대한 동적 인덱스(예: `io.in(0.U)`)

In [ ]:
class MyRoutingArbiter(numChannels: Int) extends Module {
  val io = IO(new Bundle {
    val in = Vec(numChannels, Flipped(Decoupled(UInt(8.W))))
    val out = Decoupled(UInt(8.W))
  } )

  // YOUR CODE BELOW
  ???
}

test(new MyRoutingArbiter(4)) { c =>
    // verify that the computation is correct
    // Set input defaults
    for(i <- 0 until 4) {
        c.io.in(i).valid.poke(false.B)
        c.io.in(i).bits.poke(i.U)
        c.io.out.ready.poke(true.B)
    }

    c.io.out.valid.expect(false.B)

    // Check single input valid behavior with backpressure
    for (i <- 0 until 4) {
        c.io.in(i).valid.poke(true.B)
        c.io.out.valid.expect(true.B)
        c.io.out.bits.expect(i.U)

        c.io.out.ready.poke(false.B)
        c.io.in(i).ready.expect(false.B)

        c.io.out.ready.poke(true.B)
        c.io.in(i).valid.poke(false.B)
    }

    // Basic check of multiple input ready behavior with backpressure
    c.io.in(1).valid.poke(true.B)
    c.io.in(2).valid.poke(true.B)
    c.io.out.bits.expect(1.U)
    c.io.in(1).ready.expect(true.B)
    c.io.in(0).ready.expect(false.B)

    c.io.out.ready.poke(false.B)
    c.io.in(1).ready.expect(false.B)
}

println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

Elaborating design...


<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
class MyRoutingArbiter(numChannels: Int) extends Module {
  val io = IO(new Bundle {
    val in = Vec(numChannels, Flipped(Decoupled(UInt(8.W))))
    val out = Decoupled(UInt(8.W))
  } )

  // YOUR CODE BELOW
  io.out.valid := io.in.map(\_.valid).reduce(\_ || \_)
  val channel = PriorityMux(
    io.in.map(\_.valid).zipWithIndex.map { case (valid, index) => (valid, index.U) }
  )
  io.out.bits := io.in(channel).bits
  io.in.map(\_.ready).zipWithIndex.foreach { case (ready, index) =>
    ready := io.out.ready && channel === index.U
  }
}
</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)